In [ ]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'

firefox = webdriver.Firefox(capabilities=firefox_capabilities, executable_path=geckoPath)
driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')


#firefox.get('https://www.crunchbase.com/organization/apple')
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(3)
driver.set_page_load_timeout(30)
def extract_company_techcrunch_2drivers(company):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+company
    try:
        driver.get(url)
    except:
        print('timeout of phantomjs')
        pass
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    founded = None
    employees = None
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url in url_crunchbase:
            print(company,' in crunch base')
            try:
                firefox.get(url_crunchbase)
            except:
                pass
            
            blocks_dt = []
            blocks_dd = []
            try:
                soup = BeautifulSoup(firefox.page_source, "html.parser")
                blocks_dt = soup.select("div.details dt")
                blocks_dd = soup.select("div.details dd")
            except:
                pass
            
            i = 0
            while len(blocks_dt)==0 or len(blocks_dd)==0:
                i=i+1
                if i >= 50:
                    print('fail to crawl ', company , ' in crunchbase')
                    break
                time.sleep(0.3)
                try:
                    soup = BeautifulSoup(firefox.page_source, "html.parser")
                    blocks_dt = soup.select("div.details dt")
                    blocks_dd = soup.select("div.details dd")
                except:
                    pass
                  
            for index,block in enumerate(blocks_dt):
                if 'Founded' in  block.getText() : 
                    founded = blocks_dd[index].getText()
                if 'Employees' in block.getText():
                    employees = blocks_dd[index].getText().split('|')[0]
            print(company,founded,employees)
    return founded,employees